# FIT5196 - Assignment 02

## Group 025

### Group member 1: Raguram Ramakrishnasamy Dhandapani , Student Number: 30151325
### Group member 2: Thirugnanam Ramanathan, Student Number: 30404975

#### Date: 15-09-2019

#### Version: 1.0
Environment: Python 3 and Jupyter Notebook

### Libraries Used: 
- pandas
- re
- nltk
- requests
- pdfminer
- os

## Instructions before running the file
- The stopwords-en.txt and Group025.pdf files should be present in the same folder of this file
- Necessary nltk packages should be downloaded if error throws up in the testing system (no automatic download has been setup)

## 1. Introduction

 The Main Aim of the assignment is to extract data from non-structured format and converting the extracted data into a proper format suitable for a downstream modelling task
- Input is a data-set containing 200 URLs for papers published in a popular AI conference (in PDF file format)

Output expected is divided into 2 tasks
 1. In the Sparse representation the two files:
     1. Vocabulary index file
     2. Sparse count vectors file
 2. Generating a CSV file containing three columns:
     1. Top 10 most frequent terms appearing in all Titles
     2. Top 10 most frequent Authors
     3. Top 10 most frequent terms appearing in all Abstracts

## 2.  Import libraries 

In [1]:
#Importing the necesasry libraries for analysis
import pandas as pd
import re
import requests  #To download PDF files from links provided
import pdfminer  #To convert PDF to Text format
import os

#Importing the ncessary packages from the nltk packages
import nltk     #All Text Processing actions
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.util import ngrams
from nltk.probability import *
from nltk.stem import PorterStemmer

## 3.Downloading PDF files from the Given link file

- The given pdf input file has 200 links, where each link is a pdf file
- Requests and OS packages are used to download the files and store them locally for analysis

In [2]:
#Getting the PDF files

#Extracting the List of PDFs to be downloaded
cmd = 'pdf2txt.py -o Group025.txt Group025.pdf'
os.system(cmd)


#Downloading the files
file_handle = open('Group025.txt','r')

for line in file_handle.readlines():    
    if 'PP' in line:
        line_content = line.split(' ')    
        name = line_content[0].replace('\n','')
        link = line_content[1].replace('\n','')
        
        #With the help of the requests library we are downloading the PDF files
        with open(name, "wb") as file:
            response = requests.get(link.strip())
            file.write(response.content)

print('Files downloaded')

Files downloaded


## 4.Converting PDF files to Text files

- The PDF files downloaded are non parsable i.e,non structured format
- <i><b>pdfminer</b></i> is used for converting the pdf file to the text file

In [3]:
#Converting the PDF as text files
directory = os.getcwd()

#storing the text files in the same directory
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        
        #Converting the files using the OS and PDF Miner
        txt_name = filename.replace('.pdf','.txt')    
        cmd = 'pdf2txt.py -o '+ txt_name + ' ' + filename
        os.system(cmd)
        
    else:
        continue


## 5.Text Processing and Analysis.

### PART A - Sparse Feature Generation
#### Creating Vocab list and Sparse count vectors file

### Steps Carried out
- The stop words list is created from the given stopwords
- Paper body is extracted from each file as file_content
- The file content is tokenised for every senetence
- The acronyms, words that starts with capital letters in the middle of the sentence are tokenized accordingly
- Every words are tokenised using tokenizer and tokenized words from all sentences are put together in a list
- Bigrams are extracted using Bigram Associate Measures (PMI)
- Removed independent stop words and words that have length of less than 3 in both unigrams and bigrams
- The repetitive words are removed by making the list as a set and these unique list of words from each file are added to a list

In [3]:
#Text Processing - Stage-1
directory = os.getcwd()

#Importing the stop words from the provided list
stopwords_file = open('stopwords_en.txt','r')
stopwords_list = [(word.replace('\n','')).strip() for word in stopwords_file.readlines()]

#Creating list for final analysis
unique_words_all_docs = []  #Contains the unique words of all documents
words_all_docs = []  #Contains the words from all documents
bigrams_all_docs = []  #Contains the bigrams for all documents
bigrams_count = []

#Iterating over each txt file to extract unique features
for filename in os.listdir(directory):
    if filename.endswith(".txt") and 'PP' in filename:
        
        file_handle = open(filename,'r',encoding='utf-8',errors='ignore')
        file_content = re.sub(r'-\n','',file_handle.read())
        file_content = file_content[file_content.find('1 Paper Body'):]
        file_content = file_content.replace('\n',' ')

        #Creating lists for each document analysis        
        word_tokens_swrm = [] #List to store words after stop word removal
        unique_words = [] #Contains the unique words for each document
        bigrams_doc = [] #Contains the bigrams for each document
        
        ##Sentence segmentation
        sent_seg = sent_tokenize(file_content)
        sent_seg_new = []
        
        #Except for the Capital words in beginning, the first letter of sentence is changed to lower case
        #Capital tokens in between the sentence are left in the same case
        for sent in sent_seg: 
            sent_seg_inter = (str(sent).split(' '))[0]
            if not sent_seg_inter.islower() and not sent_seg_inter.isupper():
                sent_seg_inter= sent_seg_inter.lower()
                
            sent_seg_new.append(sent_seg_inter + sent.replace((str(sent).split(' '))[0],'',1))
        
        
        ##Word tokenize
        tokenizer = RegexpTokenizer(r"[A-Za-z]\w+(?:[-'?]\w+)?",gaps = False)
        word_tokens = [tokenizer.tokenize(sent) for sent in sent_seg_new]
        
        #Making the tokens as a single list
        word_tokens = [word for slist in word_tokens for word in slist]
       
                
        #Bigrams extraction using pmi measure
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        finder = nltk.collocations.BigramCollocationFinder.from_words(word_tokens)
        bigrams_top500 = finder.nbest(bigram_measures.pmi,500)
        all_bigrams = finder.ngram_fd.items()
        
        #Having a list of all bigrams with term frequency
        bigrams_count.append(all_bigrams)
        
        ##Removing independent stopwords and words with length less than 3
        for word in word_tokens:
            if word.lower() not in stopwords_list and len(word)>=3:
                word_tokens_swrm.append(word)
        
        words_all_docs.append(word_tokens_swrm)
        
        
        #Removing independent stopwords from bigrams (bigrams and count)
        for b in bigrams_top500:
            if b[0].lower() not in stopwords_list and len(b[0].lower())>=3 and  b[1].lower() not in stopwords_list and len(b[1].lower())>=3:
                bigram = b[0]+'__'+b[1]                
                bigrams_doc.append(bigram) 
        
        bigrams_all_docs.append(bigrams_doc)
       
    
        #Unique words for this document:
        unique_words = set(word_tokens_swrm)
        unique_words_all_docs.append(unique_words)
        
print('Lists with tokens, unique words, and bigrams for analysis are created.')

Lists with tokens, unique words, and bigrams for analysis are created.


### A1.Vocabulary index file

### Steps carried out
- Frequency of all unique words is extracted for each document using FreqDist function
- Words with frequency less than 6(3%) and greater than 190(95%) are removed
- Same set of steps are performed for bigrams
- Stemming of the words is carried out using PorterStemmer
- Unigrams and Bigrams are appended as final vocabulory list and each vocab is alocated with the index
- All these vocabs are written to vocabulory file

In [8]:
###Text processing - Stage 2

## Removing tokens and bigrams with threshold and creating the VOCAB list

##TOKENS Extraction
uniquewords_all = [collec for ulist in unique_words_all_docs for collec in ulist]

doc_freq = FreqDist(uniquewords_all)
print('Total Vocabs in the documents are ', len(doc_freq.most_common()))

#Applying threshold
vocab = [word[0] for word in doc_freq.most_common() if word[1]>6 and word[1]<=190]
print('Total Vocabs after applying the threshold ',len(vocab))
      
    
##Bigram analysis
#Combining bigrams from all the documents and filtering out the threshold values

bigrams_all = [bigr for bigl in bigrams_all_docs for bigr in bigl]

bigram_freq = FreqDist(bigrams_all)

#Extracting the top 200 bigrams from the all documents if the frequency is 7-190
bigrams_top200 = [bigr[0] for bigr in bigram_freq.most_common() if bigr[1]>6 and bigr[1]<=190][:200]



##Stemming of vocabs
stemmer = PorterStemmer()

#Stemming out the words except for the capital tokens
stem_vocab=[]
for words in vocab:
    if (any(w.isupper() for w in words)):
        stem_vocab.append(words)
    else:        
        stem_vocab.append(stemmer.stem(words))
stem_vocab = set(stem_vocab)



#Combining all vocabs together
final_vocab_list = []
final_vocab_list.append(stem_vocab)
final_vocab_list.append(bigrams_top200)
final_vocabs = [vocb for vlist in final_vocab_list for vocb in vlist]
print('Final vocabs count (unigrams and bigrams) is ',len(final_vocabs))

#Sorting the vocabs
final_vocabs = sorted(final_vocabs)

#Assigning index to each vocab
final_vocabs_dict = {}
index=0
for vocab in final_vocabs:
    final_vocabs_dict[vocab] = index
    index+=1

    
#Printing out the words to the file
#Creating the VOCAB text file
vocab_file = open('Group025_vocab.txt','w',encoding='utf-8',errors='ignore')
for k,v in final_vocabs_dict.items():
    vocab_file.write(str(k)+':'+str(v))
    vocab_file.write('\n')
vocab_file.close()
print('The Vocab file is created for all UNIGRAMS and BIGRAMS with indices.\n')
print('Sample Output')
print(list(final_vocabs_dict.items())[:10])

Total Vocabs in the documents are  34083
Total Vocabs after applying the threshold  4579
Final vocabs count (unigrams and bigrams) is  3198
The Vocab file is created for all UNIGRAMS and BIGRAMS with indices.

Sample Output
[('AAAI', 0), ('ACL', 1), ('ACM', 2), ('ACM__SIGKDD', 3), ('AISTATS', 4), ('ARO', 5), ('Aaron', 6), ('Aaron__Courville', 7), ('Academic', 8), ('Academy', 9)]


### A2.Sparse count vectors file

### Steps Carried out
- Count vector file is created for writing the sparse count vectors
- Count of each unique word in each of the file is calculated (term frequency)
- The key of the corresponding word from the final vocabulary dictionary is identified
- String is appended with file name in the beginning followed by the key of vocabulary and then its frequency in the file
- Same process is carried for bigrams as well
- The string is written in the output file and same is carried for all 200 documents

In [10]:
## Text Processing - Stage 3
### Creating Sparse matrix

#Iterating over each file to get the names of each PDF file
directory = os.getcwd()
file_list = []
for filename in os.listdir(directory):
    if filename.endswith(".pdf") and 'PP' in filename:        
        file_list.append(filename)

#Opening the output file to be written
output_sparse = open('Group025_count_vectors.txt','w',encoding='utf-8',errors='ignore')

#Process of preparing the sparse matrix
for i in range(0,200):
    
    #Getting the list of words in each document, with bigrams
    word_list = words_all_docs[i]
    unique_word_list = unique_words_all_docs[i]
    bigrams_list = bigrams_count[i]
    
    #Stemming the words (total words and unique words)
    word_list_stem = [] 
    for words in word_list:
        if (any(w.isupper() for w in words)):
            word_list_stem.append(words)
        else:        
            word_list_stem.append(stemmer.stem(words))

    unique_word_list_stem = []
    for words in unique_word_list:
        if (any(w.isupper() for w in words)):
            unique_word_list_stem.append(words)
        else:        
            unique_word_list_stem.append(stemmer.stem(words))
    
    unique_word_list_stem = set(unique_word_list_stem)
    
    
    #Preparing the string to be written in the text file
    str_to_write = str(file_list[i].replace('.pdf',''))
    
    #Counting each word if it is present in the vocab
    for word in unique_word_list_stem:
        #Counting word in word list
        count = word_list_stem.count(word)
        
        #Extracting the id for the word from the vocab list
        id=''
        if word in final_vocabs_dict.keys():            
            #Getting the ID for the word
            id = final_vocabs_dict[word]
            str_to_write = str_to_write +','+ str(id) +':'+str(count)
    
    #Counting the occurence of bigrams in document
    for bigr in bigrams_list :
        #Counting word in word list
        count = bigr[1]
        bigr_text = str(bigr[0][0])+'__'+str(bigr[0][1])
        
        #Extracting the id for the word from the vocab list
        id=''
        if bigr_text in final_vocabs_dict.keys():            
            #Getting the ID for the word
            id = final_vocabs_dict[bigr_text]
            str_to_write = str_to_write +','+ str(id) +':'+str(count)
    
    output_sparse.write(str_to_write)
    output_sparse.write('\n')
    
output_sparse.close()
print('The Sparse vectors for all documents are created and stored in the file.\n')
print('Sample Output:')
print(str_to_write)

The Sparse vectors for all documents are created and stored in the file.

Sample Output:
PP7283,3186:3,2871:2,1670:1,3051:3,1341:4,2002:2,2491:3,2803:1,851:5,1170:1,643:3,2700:4,1157:2,2160:1,1590:1,89:1,2384:2,2566:1,2188:1,1985:1,1954:7,2316:4,2058:2,827:1,730:3,1677:2,1861:2,1567:7,1177:1,32:1,1351:2,2289:8,176:2,1156:5,2707:1,1322:1,1607:7,2789:1,3178:2,2301:1,2331:3,2820:1,2200:2,2389:5,1187:10,2787:3,1469:2,1798:1,610:1,991:1,2532:26,617:2,957:4,2444:2,2759:1,2208:1,2602:8,2710:1,1820:1,2081:13,3063:4,1845:1,1704:1,1101:5,2640:3,1504:2,1770:1,1097:1,2656:1,142:2,1239:1,1186:2,2795:1,1556:3,1168:1,3062:5,2673:18,2312:2,2165:1,2621:1,3032:1,2508:11,2649:1,3082:4,2050:3,2597:1,1028:1,1374:1,2300:1,934:3,1929:1,2337:4,2259:11,2822:1,1934:2,195:1,1742:6,2029:5,1413:1,1150:2,2261:5,3080:8,2288:6,3169:1,1337:1,2888:2,1319:1,2417:15,1786:1,1377:2,163:1,1295:11,1647:1,3177:1,2087:3,1981:2,2507:19,2749:1,1789:2,1902:2,1601:7,955:3,2813:2,1099:5,1154:5,1725:1,2429:1,2184:6,1333:2,3188:1,209

## PART B - Statistics Generation
### Generating a CSV file with three columns for Titles, Authors, Abstracts

### Steps carried out
- Title, Author name and abstract of each file is seperated using the appropriate regex expression
- The title is converted to the lower case and the abstract is tokenised
- The list of words of Abstract, Title and Author name are obtained and frequency distribution is performed
- The top 10 of all three is merged to the dictionary, which is then converted to data frame
- The dataframe is then converted to CSV file

**Note : In place of tie(same frequency for words), the words are arranged in alphabetical order.**

In [14]:
#Iterating over each txt file to extract unique features

directory = os.getcwd()

#Creating the lists for analysis
title_collection = []
abstract_collection = []
authors_collection = []

for filename in os.listdir(directory):
    
    if filename.endswith(".txt") and 'PP' in filename:        
        file_handle = open(filename,'r',encoding='utf-8',errors='ignore')
        file_content = re.sub(r'-\n','',file_handle.read())
        file_content = file_content.replace('\n',' ')
        
        
        #Extracting the necessary data for the analysis
        title = file_content[0:file_content.find('Authored')]
       
        authors = re.search('Authored by:(.*?)Abstract', file_content, flags=16).group()
        authors_all = authors.split('  ')[1:-1]
        
        abstract = re.findall('Abstract(.*?)1 Paper', file_content, flags=16)[0]
        
        
        #Title analysis
        title_regex = RegexpTokenizer(r"[A-Za-z]\w+(?:[-'?]\w+)?",gaps = False)
        title_tokens = title_regex.tokenize(title)
        
        for titl in title_tokens:
            if titl.lower() not in stopwords_list:
                title_collection.append(titl.lower())
        
        
        ###Abstract analysis
        ##Sentence segmentation and tokenisation
        sent_seg = sent_tokenize(abstract)
        sent_seg_new = '' 
        
        for sent in sent_seg:  
            sent_seg_inter = (str(sent).split(' '))[0]            
            if not sent_seg_inter.islower() and not sent_seg_inter.isupper():
                sent_seg_inter = sent_seg_inter.lower()           
            sent_seg_new = sent_seg_new + sent_seg_inter + sent.replace((str(sent).split(' '))[0],'',1)
        
        #Abstract tokenizer
        abs_regex = RegexpTokenizer(r"[A-Za-z]\w+(?:[-'?]\w+)?",gaps = False)
        abs_tokens = abs_regex.tokenize(sent_seg_new)
        
        #Removing independent stopwords
        for abst in abs_tokens:
            if abst.lower() not in stopwords_list:
                abstract_collection.append(abst)
                
                
        #Author analysis
        for auth in authors_all:
            authors_collection.append(auth)
        

#Final lists for all the entries
final_abs = list(FreqDist(abstract_collection).most_common(10))
final_abs = sorted(final_abs,key=lambda x:(-x[1],x[0]))
abs_terms=[]
for item in final_abs:
    abs_terms.append(item[0])
    

final_tit = list(FreqDist(title_collection).most_common(10))
final_tit = sorted(final_tit,key=lambda x:(-x[1],x[0]))
title_terms=[]
for item in final_tit:
    title_terms.append(item[0])
    

final_author = list(FreqDist(authors_collection).most_common(10))
final_author = sorted(final_author,key=lambda x:(-x[1],x[0]))
authors_list=[]
for item in final_author:
    authors_list.append(item[0])

#Appending the data to dictionary and creating a Dataframe, then to CSV
final_list = {'top10_terms_in_abstracts':abs_terms,'top10_terms_in_titles':title_terms,'top10_authors':authors_list}
df = pd.DataFrame(final_list)
print('Output\n')
print(df)
df.to_csv('Group025_stats.csv',index=False)

Output

  top10_terms_in_abstracts top10_terms_in_titles          top10_authors
0                 learning              learning         Lawrence Carin
1                     data             inference            Aarti Singh
2                    model                models     Bernhard Sch?lkopf
3                algorithm                 model  Carl Edward Rasmussen
4                   models              networks         Josh Tenenbaum
5                  problem                linear   Martin J. Wainwright
6                     show          optimization       Masashi Sugiyama
7                  methods                sparse          Nicolas Heess
8               algorithms            stochastic   Pradeep K. Ravikumar
9                 approach                  deep               Rong Jin


## 6. Summary
- Learnings from the project
    1. Steps involved in Processing the unformatted data (pdf)
    2. Use of nltk and its functions 
    3. Text Processing - Tokenisation (words and bigrams)
    4. Stemming of the words (How stemming affects the analysis and when should we stem the words)
    5. Knowledge on Sparse matrix and its use (infered)


- Further the sparse matrix generated can be used to create tf-idf weights, which can be used in model developments for further analysis (Text analytics in ML)

## 7. References


 - Check for upper case letter - https://stackoverflow.com/questions/17140408/if-statement-to-check-whether-a-string-has-a-capital-letter-a-lower-case-letter/17140466
 
 - Sorting a list - https://stackoverflow.com/questions/15371691/how-to-sort-a-dictionary-by-value-desc-then-by-key-asc